In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from morning_price import MorningPrice
from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [3]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [4]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [5]:
aa = MorningPrice(start_time, end_time)
results = aa.markets(start_time, end_time, trading)

In [6]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.239017692 13:10 Southwell (WIN)
1.239017430 13:20 Carlisle (WIN)
1.239017705 13:40 Southwell (WIN)
1.239017436 13:50 Carlisle (WIN)
1.239017712 14:10 Southwell (WIN)
1.239017442 14:20 Carlisle (WIN)
1.239017719 14:45 Southwell (WIN)
1.239017448 14:55 Carlisle (WIN)
1.239017726 15:18 Southwell (WIN)
1.239017455 15:30 Carlisle (WIN)
1.239017733 15:50 Southwell (WIN)
1.239017462 16:00 Carlisle (WIN)
1.239017739 16:20 Southwell (WIN)
1.239017744 16:27 Wolverhampton (WIN)
1.239017468 16:33 Carlisle (WIN)
1.239017750 17:00 Wolverhampton (WIN)
1.239017758 17:30 Wolverhampton (WIN)
1.239017764 18:00 Wolverhampton (WIN)
1.239017770 18:30 Wolverhampton (WIN)
1.239017776 19:00 Wolverhampton (WIN)
1.239017782 19:30 Wolverhampton (WIN)
1.239017788 20:00 Wolverhampton (WIN)
1.239017795 20:30 Wolverhampton (WIN)


In [7]:
df1 = aa.market_runner(results)
df1

,market_id,market_start_time,venue,market_type,selection_id,horse_name
0,1.239017692,2025-02-03 13:10:00,Southwell,WIN,49695107,Ideallko
1,1.239017692,2025-02-03 13:10:00,Southwell,WIN,55467911,Pitwood Road
2,1.239017692,2025-02-03 13:10:00,Southwell,WIN,39410574,Golden De Coeur
3,1.239017692,2025-02-03 13:10:00,Southwell,WIN,41740568,Zestful Hope
4,1.239017692,2025-02-03 13:10:00,Southwell,WIN,53314887,Fever Dream
...,...,...,...,...,...,...
194,1.239017795,2025-02-03 20:30:00,Wolverhampton,WIN,68686048,Nevernay
195,1.239017795,2025-02-03 20:30:00,Wolverhampton,WIN,47672163,Not Just Yet
196,1.239017795,2025-02-03 20:30:00,Wolverhampton,WIN,39813460,Sisters In The Sky
197,1.239017795,2025-02-03 20:30:00,Wolverhampton,WIN,40008704,Kensington Agent


In [8]:
df1.dtypes

market_id                    object
market_start_time    datetime64[ns]
venue                        object
market_type                  object
selection_id                  int64
horse_name                   object
dtype: object

In [9]:
market_id_list = aa.market_id_list(df1)

In [10]:
market_books1 = aa.market_books(market_id_list, trading)

In [11]:
market_books1

[[<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>],
 [<MarketBook>]]

# from here i want to automate getting the price 5 minutes before every race
then need to run through the appropiate model (turf, chase, hurdle) and place the bet

In [12]:
df2 = aa.morning_price(market_books1)

# join df1 and df2

In [13]:
df3 = aa.join(df1, df2)

In [14]:
df3.head(15)

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,last_price_traded,total_matched
0,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,49695107,Ideallko,ACTIVE,3.50,26433.83
1,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,55467911,Pitwood Road,ACTIVE,5.00,26433.83
2,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,39410574,Golden De Coeur,ACTIVE,7.00,26433.83
3,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,41740568,Zestful Hope,ACTIVE,7.60,26433.83
4,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,53314887,Fever Dream,ACTIVE,8.60,26433.83
5,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,49128885,Rajjamataz,ACTIVE,12.50,26433.83
6,1.239017692,2025-02-03 13:10:00+00:00,Southwell,WIN,51220669,West Warhorse,ACTIVE,19.50,26433.83
7,1.239017430,2025-02-03 13:20:00+00:00,Carlisle,WIN,72651334,More Coko,ACTIVE,1.22,51393.03
8,1.239017430,2025-02-03 13:20:00+00:00,Carlisle,WIN,77786681,Throatlash,ACTIVE,9.00,51393.03
9,1.239017430,2025-02-03 13:20:00+00:00,Carlisle,WIN,45549752,Zephlyn,ACTIVE,38.00,51393.03
